In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import requests
import json
API_TOKEN  = r"GRjdYvo48gh7lYepaYgUTVtknrzqdXeuBWHyQPdzlVoMC6e3874VHwQQzw6k"

In [3]:
# PUT requests to change the sales status of existiong jobs
def update_sales_status(API_TOKEN, uuid, name, status):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        "sales_status": status
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")

# PUT requests to change the contract date
def update_contract_date(API_TOKEN, uuid, name, new_date):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        'contract_date': new_date
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")
#creat df from data stored in followup
def get_df(API_TOKEN, page):
    # url = 'https://app.followupcrm.com/api/v1/file-data'
    url = f'https://app.followupcrm.com/api/v1/file-data?page={page}'
    params = {
        'api_token': API_TOKEN,
        'per_page': 15
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        # print(len(df))
        return df, json_data
    else:
        print(f"Error {response.status_code}: {response.text}")



In [6]:
%%time 
df_list = []
df, json = get_df(API_TOKEN, 1)
last_page = json['meta']['last_page']
for i in range(1,last_page+1):
    df_list.append(get_df(API_TOKEN, i)[0])

CPU times: total: 188 ms
Wall time: 58 s


In [44]:
for key, value in json.items():
    print(key)

data
links
meta


In [53]:
json['meta']['last_page']

32

In [9]:
df[df['name'] == 'API Test']['uuid']

463    b76cb886-bd27-410c-8979-cd3bde1f581a
Name: uuid, dtype: object

In [10]:
update_sales_status(API_TOKEN,'b76cb886-bd27-410c-8979-cd3bde1f581a', 'API Test', 'API Test')

In [20]:
#list of jobs that may need status updates (bidding, Submitted)
df['bid_date'] = pd.to_datetime(df['bid_date'])
status_list = ["Bidding","New Lead","Submitted"]
reduced = df[df['sales_status'].isin(status_list)]
reduced = reduced.assign(time_passed = lambda x: pd.Timestamp(date.today()) - x['bid_date'])
reduced['notify'] = reduced['time_passed'].apply(lambda y: 1 if y > pd.Timedelta('30 days 00:00:00') else 0)

,id,uuid,team_id,name,address,city,state,zip,sales_behaviours,account_manager,...,decision_process,change_orders,record_detail_custom_fields,contact.name,contact.phone,contact.fax,contact.cell,contact.email,time_passed,notify
1,1385571,f467cd5f-7595-4ed5-bf04-bb6178a05d7a,610,8081 E. Orchard Rd.,8081 E Orchard Rd,Greenwood Village,CO,80111,"[{'name': 'Next Follow up', 'date': '2024-10-2...",Stephen Ceresa,...,None,[],[],Kelly,(303) 539-7141,,,kelly@albdev.com,NaT,0
8,1446073,c7ea5596-d935-4eba-bb35-66f4b185f6ec,610,Golden Gas Station; Demo,2200 Ford St,Golden,CO,80401,"[{'name': 'Next Follow up', 'date': '2024-11-1...",None,...,None,[],[],Brent Anderson,(303) 949-0978,,,banderson@resight-ai.com,169 days,1
36,1459351,ff89c879-eea6-49f8-86e3-dec0589f3036,610,Swire Onsite,Tower Road,Denver,CO,None,"[{'name': 'Next Follow up', 'date': '2024-12-0...",None,...,None,[],[],Jacob Angelovich,(970) 732-0530,,,jacob.angelovich@whiting-turner.com,161 days,1
38,1459365,817f905f-3d1c-4e94-b65e-61e10ab4f892,610,Swire Offsite,DIA,Denver,CO,80249,"[{'name': 'Next Follow up', 'date': '2024-12-0...",None,...,None,[],[],Collin McCarty,(720) 484-4526,,,Collin.McCarty@kimley-horn.com,36 days,1
50,1465389,e117e1dc-38d7-4e18-88c6-08f644fea5ac,610,Erie Indoor Sports Complex,Bonnell Avenue & East County Line Road,Erie,CO,80516,"[{'name': 'Next Follow up', 'date': '2024-12-0...",None,...,None,[],[],Jim Modafferi,,,(720) 625-1080,jim@themodafferigroup.com,196 days,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,1665327,93684509-d86b-4859-86b4-7a0611a1d9e0,610,40th & Walnut,3963 Walnut St,Denver,CO,80205,"[{'name': 'Estimate Due', 'date': '2025-04-25 ...",None,...,None,[],[],Kenny Deemer,,,(720) 936-3314,kennydeemer@beckgroup.com,-7 days,0
469,1665333,80a45d93-9b0c-4e09-93c6-5e0ffc967706,610,40th & Walnut,3963 Walnut St,Denver,CO,80205,"[{'name': 'Estimate Due', 'date': '2025-04-25 ...",None,...,None,[],[],Kenny Deemer,,,(720) 936-3314,kennydeemer@beckgroup.com,-7 days,0
470,1665988,fc19d6d1-6849-4cad-9d98-85f47a61ec6d,610,"Lee Farm First Subdivision, Phase 1-3",None,Loveland,CO,80538,[],None,...,None,[],[],Chaz Audet,,,(720) 308-0348,chazaudet@forestar.com,-17 days,0
471,1667808,a2e3b6ec-5947-4ca4-9029-8a45b7fc7c3b,610,S Golden Rd (Site Development),South Golden Road,Golden,CO,80401,"[{'name': 'Next Follow up', 'date': '2025-04-2...",None,...,None,[],[],David Walsh,,,(765) 469-1993,dwalsh@thompsonthrift.com,-4 days,0


In [7]:
# list of jobs that need a base bid price


In [8]:
# list of jobs that need a bid date 

## Export to Excel

In [14]:
# make path for each day that the data is compiled 
end_file = f'followup_data_{str(date.today())}.xlsx'
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P5 Followup API\follow_up_data\{}".format(end_file)
# end_path
# create excel for spot checks
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
df.to_excel(writer, sheet_name="full_df")
# pivot_table.to_excel(writer, sheet_name="Pivot")
writer.close()

# data set of old budgets to become budget loss

In [ ]:
# data set of old budgets to become budget loss
df2 = df
df2['bid_date'] = pd.to_datetime(df2['bid_date'], format='%m/%d/%Y')
in_range = df2[df2['bid_date'] < pd.to_datetime("2024-12-31")]
# I only want budgets
just_buds = in_range[in_range['sales_status'] == 'Budget']
len(just_buds)
# in_range = df[df['']]
for ind in range(0,len(just_buds)):
    current = just_buds.iloc[ind]
    # print(current['uuid'], current['name'])
    update_sales_status(API_TOKEN, current['uuid'], current['name'], 'Budget Loss')
    print(ind)

# data set of won jobs to update contract date 

In [ ]:
df3 = df
df3['bid_date'] = pd.to_datetime(df3['bid_date'], format='%m/%d/%Y')
df3['dead_date'] = pd.to_datetime(df3['dead_date']).dt.date
# df3['dead_date'] = df3['dead_date'].astype('datetime64[D]')
in_range = df3[df3['bid_date'] < pd.to_datetime("2024-10-15")]
# I only want budgets
just_wins = in_range[in_range['sales_status'] == 'Won']
need_contract_date = just_wins.dropna(subset=['dead_date'])
# need_contract_date = just_wins[just_wins['dead_date'] != pd.NaT]
print(len(just_wins), len(need_contract_date))
for ind in range(0,len(need_contract_date)):
    current = need_contract_date.iloc[ind]
    # print(current['uuid'], current['name'], str(current['dead_date']))
    update_contract_date(API_TOKEN, current['uuid'], current['name'], current['dead_date'])
    print(ind)


In [7]:
df.columns

Index(['id', 'uuid', 'team_id', 'name', 'address', 'city', 'state', 'zip',
       'sales_behaviours', 'account_manager', 'estimator', 'companies',
       'proposal_delivered', 'sales_status', 'office', 'Lead_From',
       'contract_amount', 'base_bid', 'contract_date', 'contract_type',
       'bid_date', 'last_update', 'created_at', 'external_id',
       'external_vendor', 'custom_fields', 'notes', 'outlook_conversation_id',
       'estimated_contract_date', 'gross_margin_percentage', 'labor_hours',
       'dead_date', 'marketing_status', 'pre_bid', 'take_off_person',
       'products_and_services', 'client_profile', 'relate_and_connect',
       'rules_of_engagement', 'find_problems', 'money_step',
       'decision_process', 'change_orders', 'record_detail_custom_fields',
       'contact.name', 'contact.phone', 'contact.fax', 'contact.cell',
       'contact.email'],
      dtype='object')